In [1]:
import torch
import numpy as np, cv2, pandas as pd, glob, time
import matplotlib.pyplot as plt
%matplotlib inline
import torch.nn as nn
from torch import optim
import torch.nn.functional as F
from torch.utils.data import Dataset, DataLoader
import torchvision
from torchvision import transforms, models, datasets

In [2]:
device='cuda' if torch.cuda.is_available() else 'cpu'

In [3]:
#@ Fetching Datasets:
from pydrive2.auth import GoogleAuth
from pydrive2.drive import GoogleDrive
from google.colab import auth
from oauth2client.client import GoogleCredentials

auth.authenticate_user()
gauth=GoogleAuth()
gauth.credentials=GoogleCredentials.get_application_default()
drive=GoogleDrive(gauth)

def getfile(file_id, name):
  downloaded=drive.CreateFile({'id': file_id})
  downloaded.GetContentFile(name)

getfile('1Z1RqRo0_JiavaZw2yzZG6WETdZQ8qX86', 'fairface-img-margin025-trainval.zip')
getfile('1k5vvyREmHDW5TSM9QgB04Bvc8C8_7dl-', 'fairface-label-train.csv')
getfile('1_rtz1M1zhvS0d5vVoXUamnohB6cJ02iJ', 'fairface-label-val.csv')

!unzip -qq fairface-img-margin025-trainval.zip

In [4]:
trn_df = pd.read_csv('fairface-label-train.csv')
val_df = pd.read_csv('fairface-label-val.csv')
trn_df.head()

,file,age,gender,race,service_test
0,train/1.jpg,59,Male,East Asian,True
1,train/2.jpg,39,Female,Indian,False
2,train/3.jpg,11,Female,Black,False
3,train/4.jpg,26,Female,Indian,True
4,train/5.jpg,26,Female,Indian,True


In [5]:
IMAGE_SIZE=224
#@ Class for Gender and Age:
class GenderAge(Dataset):
  def __init__(self, df, tfms=None):
    self.df=df
    self.normalize=transforms.Normalize(
                     mean=[0.485, 0.456, 0.406],
                     std=[0.229, 0.224, 0.225])

  def __len__(self):
    return len(self.df)

  def __getitem__(self, ix):
    f=self.df.iloc[ix].squeeze()
    file=f.file
    gen=f.gender=='Female'
    age=f.age
    im=cv2.imread(file)
    im=cv2.cvtColor(im, cv2.COLOR_BGR2RGB)
    return im ,age, gen

  def preprocess(self, im):
    im=cv2.resize(im, (IMAGE_SIZE, IMAGE_SIZE))
    im=torch.tensor(im).permute(2, 0, 1)
    im=self.normalize(im/255.)
    return im[None]

  def collate_func(self, batch):
    'preprocess images, ages, gender'
    ims, ages, genders=[], [], []
    for im, age, gender in batch:
      im=self.preprocess(im)
      ims.append(im)

      ages.append(float(int(age)/80))
      genders.append(torch.tensor(float(gender)).to(device).float())

    ages, genders=[torch.tensor(x).to(device).float() for x in [ages, genders]]
    ims=torch.cat(ims).to(device)
    return ims, ages, gender

In [6]:
train=GenderAge(trn_df)
val=GenderAge(val_df)

In [13]:
#@ Specifying loaders:
device='cuda' if torch.cuda.is_available() else 'cpu'
train_dl=DataLoader(train, batch_size=32, shuffle=True, drop_last=True, collate_fn=train.collate_func)
test_dl=DataLoader(val, batch_size=32, collate_fn=val.collate_func)
a, b, c, = next(iter(train_dl))
print(a.shape, b.shape)

torch.Size([32, 3, 224, 224]) torch.Size([32])


In [14]:
#@ Defining model:
def get_model():
  model=models.vgg16(weights='VGG16_Weights.DEFAULT')
  for param in model.parameters():
    param.requires_grad=False

  model.avgpool=nn.Sequential(
      nn.Conv2d(512, 512, kernel_size=3),
      nn.MaxPool2d(2),
      nn.ReLU(),
      nn.Flatten()
  )

  class AgeGenderClassifier(nn.Module):
    def __init__(self):
      super(AgeGenderClassifier, self).__init__()
      self.intermediate=nn.Sequential(
                        nn.Linear(2048, 512),
                        nn.ReLU(),
                        nn.Dropout(0.3),
                        nn.Linear(512, 128),
                        nn.ReLU(),
                        nn.Dropout(0.3),
                        nn.Linear(128, 64),
                        nn.ReLU()
      )
      self.age_classifier=nn.Sequential(
          nn.Linear(64, 1),
          nn.Sigmoid()
      )

      self.gender_classifier=nn.Sequential(
          nn.Linear(64, 1),
          nn.Sigmoid()
      )

    def forward(self, x):
      x=self.intermediate(x)
      age=self.age_classifier(x)
      gender=self.gender_classifier(x)
      return gender, age

  model.classifier=AgeGenderClassifier()

  gender_criterion=nn.BCELoss()
  age_criterion=nn.L1Loss()
  loss_func=gender_criterion, age_criterion
  optimizer=torch.optim.Adam(model.parameters(), lr=1e-4)

  return model.to(device), loss_func, optimizer

model, loss_func, optimizer=get_model()

In [15]:
!pip install torchsummary

In [16]:
from torchsummary import summary
summary(model, input_size=(3, 224, 224), device=device)

----------------------------------------------------------------
        Layer (type)               Output Shape         Param #
            Conv2d-1         [-1, 64, 224, 224]           1,792
              ReLU-2         [-1, 64, 224, 224]               0
            Conv2d-3         [-1, 64, 224, 224]          36,928
              ReLU-4         [-1, 64, 224, 224]               0
         MaxPool2d-5         [-1, 64, 112, 112]               0
            Conv2d-6        [-1, 128, 112, 112]          73,856
              ReLU-7        [-1, 128, 112, 112]               0
            Conv2d-8        [-1, 128, 112, 112]         147,584
              ReLU-9        [-1, 128, 112, 112]               0
        MaxPool2d-10          [-1, 128, 56, 56]               0
           Conv2d-11          [-1, 256, 56, 56]         295,168
             ReLU-12          [-1, 256, 56, 56]               0
           Conv2d-13          [-1, 256, 56, 56]         590,080
             ReLU-14          [-1, 256,

In [25]:
def train_batch(data, model, optimizer, criteria):
  model.train()
  ims, age, gender=data
  optimizer.zero_grad()
  pred_gender, pred_age=model(ims)
  gender_criterion, age_criterion=criteria
  gender_loss=gender_criterion(pred_gender.squeeze(), gender)
  age_loss=age_criterion(pred_age.squeeze(), age)
  total_loss=gender_loss + age_loss
  total_loss.backward()
  optimizer.step()
  return total_loss

def validate_batch(data, model, criteria):
  model.eval()
  ims, age, gender=data
  with torch.no_grad():
    pred_gender, pred_age=model(ims)
    gender_criterion, age_criterion=criteria
    gender_loss=gender_criterion(pred_gender().squeeze(), gender)
    age_loss=age_criterion(pred_age.squeeze(), age)
    total_loss=gender_loss + age_loss
    pred_gender=(pred_gender > 0.5).squeeze()
    gender_accuracy=(pred_gender==gender).float().sum()
    age_mae=torch.abs(age=pred_age).float().sum()
    return total_loss, gender_accuracy, age_mae

In [26]:
#@ Training:
import time
model, criteria, optimizer=get_model()
val_gender_accuracies=[]
val_age_maes=[]
train_losses=[]
val_losses=[]

n_epochs=5
best_test_loss=1000
start=time.time()

for epoch in range(n_epochs):
  epoch_train_loss, epoch_test_loss=0, 0
  val_age_mae, val_gender_acc, ctr=0,0,0
  _n=len(train_dl)
  for ix, data in enumerate(train_dl):
        # if ix == 100: break
        loss = train_batch(data, model, optimizer, criteria)
        epoch_train_loss += loss.item()

  for ix, data in enumerate(test_dl):
        # if ix == 10: break
        loss, gender_acc, age_mae = validate_batch(data, model, criteria)
        epoch_test_loss += loss.item()
        val_age_mae += age_mae
        val_gender_acc += gender_acc
        ctr += len(data[0])

  val_age_mae /= ctr
  val_gender_acc /= ctr
  epoch_train_loss /= len(train_dl)
  epoch_test_loss /= len(test_dl)

  elapsed = time.time()-start
  best_test_loss = min(best_test_loss, epoch_test_loss)
  print('{}/{} ({:.2f}s - {:.2f}s remaining)'.format(epoch+1, n_epochs, time.time()-start, (n_epochs-epoch)*(elapsed/(epoch+1))))
  info = f'''Epoch: {epoch+1:03d}\tTrain Loss: {epoch_train_loss:.3f}\tTest: {epoch_test_loss:.3f}\tBest Test Loss: {best_test_loss:.4f}'''
  info += f'\nGender Accuracy: {val_gender_acc*100:.2f}%\tAge MAE: {val_age_mae:.2f}\n'
  print(info)

  val_gender_accuracies.append(val_gender_acc)
  val_age_maes.append(val_age_mae)


AttributeError: 'bool' object has no attribute 'size'